In [2]:
import sys, os, pprint, glob, re

In [3]:
IN_PATH = './in.txt'

f = open(IN_PATH, 'r')
buf = f.readlines()
f.close()
pprint.pprint(buf)

['AAA O\n',
 'BBBBB O\n',
 'CC B-GOODSNAME 1_2\n',
 'DDD I-GOODSNAME 1_2\n',
 'EEE I-GOODSNAME 1_2\n',
 'FFFFF E-GOODSNAME 1_2\n',
 'G O\n',
 'H O\n',
 'I O\n',
 'J O\n',
 'KKK O\n',
 'LL O\n',
 'MMMM S-ORDERCURRENCY 1\n',
 'NNNNN S-ORDERPRICE 1\n',
 'OOOO B-ORDERUNIT 1\n',
 'PPPPPP E-ORDERUNIT 1\n',
 'QQQQQQQQQQQQ O\n',
 'RR O\n',
 'SS O\n',
 'TT O\n',
 'UU O\n',
 'V S-ORDERCURRENCY 2\n',
 'W S-ORDERUNIT 2\n',
 'X O\n',
 'YYY B-ORDERUNIT 3\n',
 'ZZZZZZZZZZZZZZZZ E-ORDERUNIT 3']


In [4]:
outs = []
pre_word = ''
pre_label = ''

word_sep = '_'
item_sep = ' ' 

for i, data in enumerate(buf):
    
    lists = re.sub(r'\n$', '', data).split(' ')
    
    if len(lists) == 2:
        (word, label) = lists
        relation = ''
    elif len(lists) == 3:
        (word, label, relation) = lists
    else:
        print('illegal data line:{}'.format(i + 1))
        continue

    if label == 'O':
        outs.append(word + item_sep + 'O' + item_sep + relation)
    elif label.startswith('S-'):
        outs.append(word + item_sep + label[2:] + item_sep + relation)
    elif label.startswith('B-'):
        pre_word = word
    elif label.startswith('I-'):
        pre_word = pre_word + word_sep + word
    elif label.startswith('E-'):
        pre_word = pre_word + word_sep + word
        outs.append(pre_word + item_sep + label[2:] + item_sep + relation)
    else:
        pass
    
pprint.pprint(outs)

['AAA O ',
 'BBBBB O ',
 'CC_DDD_EEE_FFFFF GOODSNAME 1_2',
 'G O ',
 'H O ',
 'I O ',
 'J O ',
 'KKK O ',
 'LL O ',
 'MMMM ORDERCURRENCY 1',
 'NNNNN ORDERPRICE 1',
 'OOOO_PPPPPP ORDERUNIT 1',
 'QQQQQQQQQQQQ O ',
 'RR O ',
 'SS O ',
 'TT O ',
 'UU O ',
 'V ORDERCURRENCY 2',
 'W ORDERUNIT 2',
 'X O ',
 'YYY_ZZZZZZZZZZZZZZZZ ORDERUNIT 3']


In [5]:
str_out = ''
word_sep = ' '

for i, data in enumerate(outs):
    
    (word, label, relation) = re.sub(r'\n$', '', data).split(' ')
    
    if label != 'O':
        tag1 = r'<' + label + r'>'
        tag2 = r'</' + label + r'>'
        str_out = str_out + word_sep + tag1 + word + tag2
    else:
        str_out = str_out + word_sep + word

print(str_out)

 AAA BBBBB <GOODSNAME>CC_DDD_EEE_FFFFF</GOODSNAME> G H I J KKK LL <ORDERCURRENCY>MMMM</ORDERCURRENCY> <ORDERPRICE>NNNNN</ORDERPRICE> <ORDERUNIT>OOOO_PPPPPP</ORDERUNIT> QQQQQQQQQQQQ RR SS TT UU <ORDERCURRENCY>V</ORDERCURRENCY> <ORDERUNIT>W</ORDERUNIT> X <ORDERUNIT>YYY_ZZZZZZZZZZZZZZZZ</ORDERUNIT>


### O-tag

`outs:
['AAA O ',
 'BBBBB O ',
 'CC_DDD_EEE_FFFFF GOODSNAME 1_2',
 'G O ',
 'H O ',
 'I O ',
 'J O '
]`

In [35]:
# O-tag のインデックスを得る
#pprint.pprint(outs)
list_Oidx = []
for i, tmp in enumerate(outs, 1):
    (word, label, relation) = re.sub(r'\n$', '', tmp).split(' ')
    if label != 'O':
        continue
    list_Oidx.append(i)
pprint.pprint(list_Oidx)

[1, 2, 4, 5, 6, 7, 8, 9, 13, 14, 15, 16, 17, 20]


`list_Oidx
[1, 2, 4, 5, 6, 7, 8, 9, 13, 14, 15, 16, 17, 20]`

In [50]:
# O-tag のインデックス(番号連結)を得る
list_Oidx_con = []
for i in list_Oidx:
    pre_idx =i
    list_tmp = []
    list_tmp2 = []
    for j in list_Oidx:
        if j < i:
            continue
        if j - pre_idx > 1:
            break
        list_tmp.append(j)
        pre_idx = j
    for k in list_tmp:
        list_tmp2.append(k)
        list_Oidx_con.append(list_tmp2.copy())
        
pprint.pprint(list_Oidx_con)

[[1],
 [1, 2],
 [2],
 [4],
 [4, 5],
 [4, 5, 6],
 [4, 5, 6, 7],
 [4, 5, 6, 7, 8],
 [4, 5, 6, 7, 8, 9],
 [5],
 [5, 6],
 [5, 6, 7],
 [5, 6, 7, 8],
 [5, 6, 7, 8, 9],
 [6],
 [6, 7],
 [6, 7, 8],
 [6, 7, 8, 9],
 [7],
 [7, 8],
 [7, 8, 9],
 [8],
 [8, 9],
 [9],
 [13],
 [13, 14],
 [13, 14, 15],
 [13, 14, 15, 16],
 [13, 14, 15, 16, 17],
 [14],
 [14, 15],
 [14, 15, 16],
 [14, 15, 16, 17],
 [15],
 [15, 16],
 [15, 16, 17],
 [16],
 [16, 17],
 [17],
 [20]]


`list_Oidx_con
[[1],
 [1, 2],
 [2],
 [4],
 [4, 5],
 [4, 5, 6],
 [4, 5, 6, 7],
 [4, 5, 6, 7, 8],
 [4, 5, 6, 7, 8, 9],
 [5],
`

In [154]:
import random
print('len:{}'.format(len(list_Oidx_con)))
rnd = random.randint(0, len(list_Oidx_con) - 1)
print('rnd:{}'.format(rnd))
print(list_Oidx_con[rnd])
print(type(list_Oidx_con[rnd]))

len:40
rnd:9
[5]
<class 'list'>
